In [1]:
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor, as_completed
from ras_commander import init_ras_project, RasExamples, RasCmdr, RasPlan, RasHdf, RasUtils

In [2]:
# Define versions to compare
versions = ['6.6', '6.5', '6.4.1', '6.3.1', '6.2', '6.1', '5.0.7']

# Extract BaldEagleCrkMulti2D project
ras_examples = RasExamples()
project_path = ras_examples.extract_project(["BaldEagleCrkMulti2D"])[0]

2024-10-07 16:34:37,515 - ras_commander.RasExamples - INFO - Example projects folder: c:\AutoRAS_Temp\Benchmarking 6.6\example_projects
2024-10-07 16:34:37,516 - ras_commander.RasExamples - INFO - Found zip file: c:\AutoRAS_Temp\Benchmarking 6.6\Example_Projects_6_5.zip
2024-10-07 16:34:37,516 - ras_commander.RasExamples - INFO - Loading project data from CSV...
2024-10-07 16:34:37,518 - ras_commander.RasExamples - INFO - Loaded 66 projects from CSV. Use list_categories() and list_projects() to explore them.
2024-10-07 16:34:37,518 - ras_commander.RasExamples - INFO - ----- RasExamples Extracting Project -----
2024-10-07 16:34:37,518 - ras_commander.RasExamples - INFO - Extracting project 'BaldEagleCrkMulti2D'
2024-10-07 16:34:37,519 - ras_commander.RasExamples - INFO - Project 'BaldEagleCrkMulti2D' already exists. Deleting existing folder...
2024-10-07 16:34:37,553 - ras_commander.RasExamples - INFO - Existing folder for project 'BaldEagleCrkMulti2D' has been deleted.
2024-10-07 16:34

In [3]:
# Get all plan numbers
ras_project = init_ras_project(project_path, "6.5")
print(ras_project)
plan_numbers = ras_project.plan_df['plan_number'].tolist()
print(plan_numbers)

2024-10-07 16:34:38,687 - ras_commander.RasPrj - INFO - HEC-RAS executable found at default path: C:\Program Files (x86)\HEC\HEC-RAS\6.5\Ras.exe
2024-10-07 16:34:38,687 - ras_commander.RasPrj - INFO - Initializing global 'ras' object via init_ras_project function.
2024-10-07 16:34:38,700 - ras_commander.RasPrj - INFO - Initialization complete for project: BaldEagleDamBrk
2024-10-07 16:34:38,700 - ras_commander.RasPrj - INFO - Plan entries: 11, Flow entries: 0, Unsteady entries: 10, Geometry entries: 10, Boundary conditions: 51
2024-10-07 16:34:38,700 - ras_commander.RasPrj - INFO - Project initialized. ras_instance project folder: c:\AutoRAS_Temp\Benchmarking 6.6\example_projects\BaldEagleCrkMulti2D


['13', '15', '17', '18', '19', '03', '04', '02', '01', '05', '06']


In [7]:
import time
from ras_commander import RasGeo

def run_simulation(version, plan_number):
    # Initialize project for the specific version
    ras_project = init_ras_project(project_path, str(version))
    
    # Clear geometry preprocessor files
    plan_path = RasPlan.get_plan_path(plan_number, ras_object=ras_project)
    RasGeo.clear_geompre_files(plan_path, ras_object=ras_project)
    
    # Set number of cores to 6
    RasPlan.set_num_cores(plan_number, "6", ras_object=ras_project)
    
    # Ensure geometry preprocessing is done
    RasPlan.update_plan_value(plan_number, "Run HTab", 1, ras_object=ras_project)
    
    # Compute the plan
    start_time = time.time()
    success = RasCmdr.compute_plan(plan_number, ras_object=ras_project)
    total_time = time.time() - start_time
    
    if success:
        # Get HDF file path
        hdf_path = RasPlan.get_results_path(plan_number, ras_object=ras_project)
        
        # Extract data from HDF file
        runtime_data = RasHdf.get_runtime_data(hdf_path, ras_object=ras_project)
        
        # Extract required information
        preprocessor_time = runtime_data['Preprocessing Geometry (hr)'].values[0]
        unsteady_compute_time = runtime_data['Unsteady Flow Computations (hr)'].values[0]
        
        # Get volume accounting data
        volume_accounting = RasHdf.get_group_attributes_as_df(hdf_path, "Results/Unsteady/Summary/Volume Accounting/Volume Accounting 2D", ras_object=ras_project)
        volume_error = volume_accounting['Volume Error (%)'].values[0]
        
        return {
            'Version': version,
            'Plan': plan_number,
            'Preprocessor Time (hr)': preprocessor_time,
            'Unsteady Compute Time (hr)': unsteady_compute_time,
            'Volume Error (%)': volume_error,
            'Total Time (hr)': total_time / 3600  # Convert seconds to hours
        }
    else:
        return None


# Run simulations for all versions and plans sequentially
results = []
for version in versions:
    for plan in plan_numbers:
        print(f"Running simulation for Version {version}, Plan {plan}")
        result = run_simulation(version, plan)
        if result:
            results.append(result)
            print(f"Completed: Version {version}, Plan {plan}")
        else:
            print(f"Failed: Version {version}, Plan {plan}")

# Create DataFrame from results
df = pd.DataFrame(results)

# Save initial results to CSV
df.to_csv('save_initial_results.csv', index=False)

print("Initial results saved to 'save_initial_results.csv'")

Running simulation for Version 6.6, Plan 13


UnboundLocalError: cannot access local variable 'default_path' where it is not associated with a value

In [ ]:
# Calculate averages across plans for each version
df_avg = df.groupby('Version').mean().reset_index()

# Create line graphs
plt.figure(figsize=(12, 6))

# Unsteady Runtime vs Version
plt.subplot(1, 2, 1)
plt.plot(df_avg['Version'], df_avg['Unsteady Compute Time (hr)'], marker='o')
plt.title('Average Unsteady Runtime vs HEC-RAS Version')
plt.xlabel('HEC-RAS Version')
plt.ylabel('Unsteady Runtime (hours)')

# Volume Error vs Version
plt.subplot(1, 2, 2)
plt.plot(df_avg['Version'], df_avg['Volume Error (%)'], marker='o')
plt.title('Average Volume Error vs HEC-RAS Version')
plt.xlabel('HEC-RAS Version')
plt.ylabel('Volume Error (%)')

plt.tight_layout()
plt.show()

# Save results to CSV
df.to_csv('hecras_version_comparison.csv', index=False)
df_avg.to_csv('hecras_version_comparison_averages.csv', index=False)

print("Results saved to 'hecras_version_comparison.csv' and 'hecras_version_comparison_averages.csv'")
print("Graphs have been displayed. Please check the output.")